In [75]:
from util import get_num_lines, get_vocab, embed_sequence, get_word2idx_idx2word, get_embedding_matrix
from util import TextDatasetWithGloveElmoSuffix as TextDataset
from util import evaluate
from model import RNNSequenceClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader

import csv
import h5py
import random
import math
import numpy as np
import matplotlib

from allennlp.nn.util import sort_batch_by_length, last_dim_softmax
from torch.nn.utils.rnn import pack_padded_sequence
from torch.nn.utils.rnn import pad_packed_sequence

matplotlib.use('Agg')  # to avoid the error: _tkinter.TclError: no display name and no $DISPLAY environment variable
matplotlib.use('tkagg')  # to display the graph on remote server
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
print("PyTorch version:")
print(torch.__version__)
print("GPU Detected:")
print(torch.cuda.is_available())
using_GPU = True

PyTorch version:
1.2.0
GPU Detected:
True


In [16]:
Gutenberg = []
length = []
with open('../Poetry/corpus_to_be_labeled.csv') as t:
    lines = csv.reader(t)
    next(lines)
    for line in lines:
        # Gutenberg.append([line[1].strip(), int(float(line[2])), int(float(line[3]))])
        Gutenberg.append([line[1].strip(), int(line[2])])
        words = line[1].split(' ')
        length.append(len(words))
    print(length)

print('Poetry dataset size: ', len(Gutenberg))

[6, 6, 8, 7, 9, 8, 13, 6, 7, 8, 8, 5, 8, 7, 7, 6, 7, 5, 7, 6, 7, 8, 8, 7, 9, 9, 5, 8, 5, 7, 4, 8, 7, 6, 6, 5, 7, 6, 7, 7, 8, 12, 6, 7, 9, 5, 10, 7, 5, 6, 8, 13, 5, 6, 4, 7, 7, 7, 7, 4, 7, 5, 8, 5, 10, 7, 6, 5, 7, 10, 7, 8, 8, 9, 4, 7, 9, 10, 6, 6, 5, 6, 7, 9, 6, 8, 8, 8, 7, 5, 5, 7, 6, 5, 10, 3, 9, 9, 8, 3]
Poetry dataset size:  100


In [40]:
# vocab is a set of words
vocab = get_vocab(Gutenberg)
# two dictionaries. <PAD>: 0, <UNK>: 1
word2idx, idx2word = get_word2idx_idx2word(vocab)
# glove_embeddings a nn.Embeddings
glove_embeddings = get_embedding_matrix(word2idx, idx2word, normalization=False)
# elmo_embeddings
elmos_poetry = None
# suffix_embeddings: number of suffix tag is 2, and the suffix embedding dimension is 50
suffix_embeddings = nn.Embedding(2, 50)


vocab size:  491


100%|██████████| 2196017/2196017 [00:29<00:00, 74560.88it/s]

Number of pre-trained word vectors loaded:  338
Embeddings mean:  -0.0036560872104018927
Embeddings stdev:  0.35528913140296936


In [56]:
embedded_Gutenberg = [[embed_sequence(example[0], example[1], word2idx, glove_embeddings,
                                   elmos_poetry, suffix_embeddings)]
                   for example in Gutenberg]
print('embedded_Gutenberg size: ', len(embedded_Gutenberg))
# print(embedded_Gutenberg)

embedded_Gutenberg size:  100


In [52]:
# sentences = [example[0] for example in embedded_Gutenberg]
# print(sentences)

In [51]:
# dataloader_sentences = TextDataset(sentences)
# print(dataloader_sentences)
# poetry = DataLoader(dataset=dataloader_sentences, batch_size = 10, shuffle=True, collate_fn=TextDataset.collate_fn)
# print(poetry)

In [26]:
net = torch.load('../models/LSTMSuffixElmoAtt_Poetry_fold_0_epoch_23.pt')
net.eval()

RNNSequenceClassifier(
  (rnn): LSTM(350, 300, batch_first=True, bidirectional=True)
  (attention_weights): Linear(in_features=600, out_features=1, bias=True)
  (output_projection): Linear(in_features=600, out_features=2, bias=True)
  (dropout_on_input_to_LSTM): Dropout(p=0.2, inplace=False)
  (dropout_on_input_to_linear_layer): Dropout(p=0, inplace=False)
)

In [82]:
print(len(embedded_Gutenberg))
# # embedded_Gutenberg = torch.FloatTensor(embedded_Gutenberg)
# # embedded_Gutenberg = np.array(embedded_Gutenberg)
# embedded_Gutenberg = pd.DataFrame(embedded_Gutenberg)
embedded_Gutenberg.astype(float)
embedded_Gutenberg = torch.from_numpy(embedded_Gutenberg)
print(embedded_Gutenberg)

100


ValueError: setting an array element with a sequence.

In [70]:
predict = net(embedded_Gutenberg, length)

TypeError: dropout(): argument 'input' (position 1) must be Tensor, not numpy.ndarray